In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Dropout, Flatten, Activation, Lambda, Input
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers.core import Lambda
from keras import initializers
from keras.utils import get_file
import tensorflow as tf
import numpy as np
import os

In [ ]:
import collections

data_path = '../data/yearbook/'

f = open(data_path + 'yearbook_train.txt', 'r')

freq = {};
normal_const = 0;

for line in f:
    line = line.rstrip()
    image, year = line.split("\t")
    if year in freq:
        freq[year] += 1
    else:
        freq[year] = 1

normal_const = np.sum(freq.values())
for key in freq:
    freq[key] = freq[key]/float(normal_const);
    
sorted_freq = collections.OrderedDict(sorted(freq.items()))

idx = 0;
class_weights_train = {}
idx2year = {}

for key in sorted_freq:
    class_weights_train[idx] = sorted_freq[key]
    idx2year[idx] = key
    idx += 1

In [ ]:
import math
import os
import numpy as np
from keras.preprocessing.image import Iterator
from keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing.image import apply_transform, transform_matrix_offset_center
import keras.backend as K
from keras.preprocessing.image import random_rotation, random_shear, random_shift, random_zoom
from skimage import exposure

class RegressDataGen:
    def __init__(self, directory, map_file, target_size = (171, 186, 3), 
                 class_weights_train = None, multi_output=False, do_augmentation=True, 
                 samplewise_center = True,
                 samplewise_std_deviation = True,
                 multi_input=False
                ):
        self.directory = directory
        self.map_file = map_file
        self.filenames = []
        self.map = {}
        self.fnameToGender = {}
        self.target_size = target_size
        self.populate_filenames()
        self.populate_mapping()
        self.regressIter = None
        self.steps = 0
        self.samplewise_center = samplewise_center
        self.samplewise_std_deviation = samplewise_std_deviation
        self.height_shift_range = 0.2
        self.width_shift_range = 0.2
        self.max_rotation = 45
        self.shear = 0.785398
        self.zoom_range = (0.5, 0.5)
        self.do_augmentation = do_augmentation
        self.class_weights_train = class_weights_train
        self.equalizehist = False
        self.multi_output = multi_output
        self.multi_input = multi_input
        self.lastN = []
        
    def _recursive_list(self, subpath):
        return sorted(
            os.walk(subpath, followlinks=False), key=lambda tpl: tpl[0])
    
    def populate_mapping(self):
        f = open(self.map_file, 'r')

        for line in f:
            line = line.rstrip()
            image, year = line.split("\t")
            gender, imfilename = image.split("/")
            if gender is 'M':
                encodeGender = 1
            elif gender is 'F':
                encodeGender = 0
            self.fnameToGender[image] = encodeGender
            self.map[image] = year
            
    def populate_filenames(self):
        base_dir = self.directory
        for root, _, files in self._recursive_list(base_dir):
            for fname in files:
                if fname.lower().endswith('.' + 'png'):
                    self.filenames.append(os.path.relpath(os.path.join(root, fname), base_dir))
                    
    def preprocess(self, x):
        if self.equalizehist:
            x = exposure.equalize_hist(x)

        return x
            
    def augment_data(self, x):
        
        x = random_shift(x, self.width_shift_range, self.height_shift_range, 
                         row_axis=0, col_axis = 1, channel_axis = 2)
        x = random_rotation(x, self.max_rotation, 
                            row_axis = 0, col_axis = 1, channel_axis = 2)
        x = random_shear(x, self.shear, row_axis = 0, col_axis = 1, channel_axis = 2)
        x = random_zoom(x, self.zoom_range, row_axis = 0, col_axis = 1, channel_axis = 2)
        
        return x
            
    def flow_from_directory(self, batch_size = 32, shuffle = True, seed = 42):
        
        self.regressIter = Iterator(len(self.filenames), batch_size = batch_size, shuffle = shuffle, seed = seed)
        
        if self.do_augmentation:
            factor = 3
        else:
            factor = 1
        
        self.steps = math.ceil(len(self.filenames)/batch_size) * factor
        
        return self
    
    def next(self, *args, **kwargs):
           
        self.lastN = []
        
        idx_array, cur_idx, bs = next(self.regressIter.index_generator)
        
        batch_x = np.zeros(tuple([len(idx_array)] + list(self.target_size)), dtype=K.floatx())
        
        batch_y = np.zeros(tuple([len(idx_array)]), dtype=K.floatx())
        
        if self.multi_output:
            batch_y_gender = np.zeros(tuple([len(idx_array)]), dtype=K.floatx())
    
        if self.multi_input:
            batch_x_gender = np.zeros(tuple([len(idx_array)]), dtype=K.floatx())
        
        if self.class_weights_train is not None:
            sample_weights = np.ones(tuple([len(idx_array)]), dtype=K.floatx())
        
        for i, j in enumerate(idx_array):
            fname = self.filenames[j]
            self.lastN.append(fname)
            img = load_img(
                  os.path.join(self.directory, fname),
                  grayscale = False,
                  target_size= self.target_size)
            x = np.array(img_to_array(img, data_format='channels_last'))
            x = self.preprocess(x)
            batch_x[i] = x
            batch_y[i] = self.map[fname]
            
            if self.multi_output:
                batch_y_gender[i] = self.fnameToGender[fname]
            
            if self.multi_input:
                batch_x_gender[i] = self.fnameToGender[fname]
            
            if self.class_weights_train is not None:
                if self.multi_output:
                    sample_weights[i] = self.class_weights_train[batch_y[i].astype('int').astype('str')]
                else:
                    sample_weights[i] = self.class_weights_train[batch_y[i].astype('int').astype('str')]
        
        if self.samplewise_center:
            for x in batch_x:
                x -= np.mean(x)
        
        if self.samplewise_std_deviation:
            for x in batch_x:
                x /= np.std(x)
        
        if self.do_augmentation:
            for x in batch_x:
                x = self.augment_data(x)
        
        if self.multi_output:
            if self.class_weights_train is not None:
                return batch_x, {'out_year' : batch_y, 'out_gender': batch_y_gender}, {'out_year' : sample_weights, 'out_gender' : sample_weights} 
            else:
                return batch_x, {'out_year' : batch_y, 'out_gender': batch_y_gender}
            
        elif self.multi_input:
            if self.class_weights_train is not None:
                return {'input_1' : batch_x, 'input_2': batch_x_gender}, batch_y, sample_weights
            else:
                return {'input_1' : batch_x, 'input_2': batch_x_gender}, batch_y
        else:    
            if self.class_weights_train is not None:
                return (batch_x, batch_y, sample_weights)
            else:
                return (batch_x, batch_y)

In [ ]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.layers import Input
from keras.models import Model
from keras.applications.xception import Xception

train = RegressDataGen(data_path + 'train',
                       data_path + 'yearbook_train.txt', 
                       class_weights_train = sorted_freq,
                       do_augmentation = False,
                      )
valid = RegressDataGen(data_path + 'valid',
                       data_path + 'yearbook_valid.txt',
                       class_weights_train = sorted_freq, 
                       do_augmentation = False,
                      )

train = train.flow_from_directory()
valid = valid.flow_from_directory(shuffle=False)

mean_value = 0
for key in freq:
    mean_value += freq[key] * float(key)
    
pretrained_model = Xception(include_top=False, weights='imagenet', input_shape=(171, 186, 3))
x = pretrained_model.output
x = Conv2D(16, (1, 1), activation='relu')(x)
x = Flatten()(x)
x = Dense(32, activation='relu', bias_initializer=keras.initializers.Ones())(x)
predicted_year = Dense(1, bias_initializer = keras.initializers.Constant(mean_value))(x)

model = Model(inputs=pretrained_model.input, outputs=predicted_year)

lr = 1e-3
def lr_schedule(epoch):
    return lr * (0.1 ** float(epoch / 10.0))

model.compile(Adam(lr=lr), loss='mae', metrics=['mae'])

for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
training = False
num_experiment = 12
if training:
    with tf.device('/gpu:0'):
        filename = "will_{}.h5".format(num_experiment)
        print ("Starting experiment " + str(num_experiment))
        model.fit_generator(train, steps_per_epoch = train.steps, epochs = 20,                                
                                   validation_data = valid, 
                                   validation_steps = valid.steps,
                                   callbacks=[LearningRateScheduler(lr_schedule),
                                ModelCheckpoint(data_path + filename, save_best_only=True)]
                           )
        print("Saved " + filename)
        
else:
    model.load_weights(data_path + 'will_{}.h5'.format(num_experiment))
    print(model.metrics_names)
    print(model.evaluate_generator(valid, valid.steps))

## Xception experiments

### Warning: Experiment 6, 7, 8 had bad learning rates

will_6.h5:
- loss = mse
- epochs = 20 without data augmentation
- lr = 1e-1 with lr decay
- conv 1x1 = 16 filers
- dense = 16

Epoch 8/20
713/713 [==============================] - 640s - loss: 6.6840 - mean_absolute_error: 20.0524 - val_loss: 3.9468 - val_mean_absolute_error: 16.9953

will_7.h5:
- loss = mae
- epochs = 20 without data augmentation
- lr = 1e-1 with lr decay
- conv 1x1 = 16 filers
- dense = 16

Epoch 3/20
713/713 [==============================] - 642s - loss: 0.2597 - mean_absolute_error: 19.9808 - val_loss: 0.1756 - val_mean_absolute_error: 17.1463


will_8.h5:
- loss = mse
- epochs = 20 without data augmentation
- lr = 1e-1 with lr decay
- conv 1x1 = 32 filers
- dense = 64

Epoch 5/20
713/713 [==============================] - 640s - loss: 6.6749 - mean_absolute_error: 20.0370 - val_loss: 3.9561 - val_mean_absolute_error: 16.9929


will_9.h5:
- loss = mse
- epochs = 20 without data augmentation
- lr = 1e-3 with lr decay
- conv 1x1 = 32 filers
- dense = 64

713/713 [==============================] - 640s - loss: 0.0052 - mean_absolute_error: 0.5249 - val_loss: 0.2931 - val_mean_absolute_error: 3.6388

will_10.h5:
- loss = mse
- epochs = 20 without data augmentation
- lr = 1e-3 with lr decay
- conv 1x1 = 16 filers
- dense = 32

713/713 [==============================] - 648s - loss: 0.0275 - mean_absolute_error: 1.1997 - val_loss: 0.2914 - val_mean_absolute_error: 3.5716

will_11.h5:
- loss = mse
- epochs = 6 wita data augmentation * 3
- lr = 1e-3 with lr decay
- conv 1x1 = 16 filers
- dense = 32

2139/2139 [==============================] - 1847s - loss: 0.0788 - mean_absolute_error: 1.8316 - val_loss: 0.3522 - val_mean_absolute_error: 3.9830

will_12.h5:
- loss = mae
- epochs = 20 without data augmentation
- lr = 1e-3 with lr decay
- conv 1x1 = 16 filers
- dense = 32

713/713 [==============================] - 635s - loss: 0.0057 - mean_absolute_error: 0.6751 - val_loss: 0.0363 - val_mean_absolute_error: 3.4857

will_13.h5:
- loss = mse
- epochs = 20 without data augmentation
- lr = 1e-3 with lr decay
- conv 1x1 = 8 filers
- dense = 16

713/713 [==============================] - 640s - loss: 0.0076 - mean_absolute_error: 0.6360 - val_loss: 0.2842 - val_mean_absolute_error: 3.5779


## VGG19 experiments

will_1.h5:
- loss = mse
- epochs = 3
- lr = 1e-2
- conv 1x1 = 64 filers
- dense = 128

Epoch 1/3
2139/2139 [==============================] - 1777s - loss: 1.7294 - acc: 0.0433 - mean_absolute_error: 8.9744 - val_loss: 1.2350 - val_acc: 0.0403 - val_mean_absolute_error: 8.3663

Epoch 2/3
2139/2139 [==============================] - 1706s - loss: 1.0568 - acc: 0.0573 - mean_absolute_error: 6.9857 - val_loss: 1.3283 - val_acc: 0.0404 - val_mean_absolute_error: 8.6860

Epoch 3/3
2139/2139 [==============================] - 1704s - loss: 0.8709 - acc: 0.0644 - mean_absolute_error: 6.3560 - val_loss: 0.9722 - val_acc: 0.0547 - **val_mean_absolute_error: 7.2192**

will_2.h5:
- loss = mse
- epochs = 1
- lr = 3e-2
- conv 1x1 = 32 filers
- dense = 64

Epoch 1/1
2139/2139 [==============================] - 1718s - loss: 1.9574 - acc: 0.0403 - mean_absolute_error: 9.6086 - val_loss: 1.6574 - val_acc: 0.0375 - **val_mean_absolute_error: 9.5908**

will_3.h5:
- loss = mse
- epochs = 3 without data augmentation
- lr = 3e-2
- conv 1x1 = 32 filers
- dense = 64


Epoch 2/3
713/713 [==============================] - 388s - loss: 1.8113 - acc: 0.0436 - mean_absolute_error: 9.3031 - val_loss: 1.4613 - val_acc: 0.0378 - **val_mean_absolute_error: 9.0060**

will_4.h5:
- loss = mse
- epochs = 20 without data augmentation
- lr = 3e-2
- conv 1x1 = 50 filers
- dense = 100

Epoch 19/20
713/713 [==============================] - 387s - loss: 0.7039 - acc: 0.0581 - mean_absolute_error: 5.7438 - val_loss: 0.9351 - val_acc: 0.0663 - **val_mean_absolute_error: 6.9641**

will_5.h5:
- loss = mse
- epochs = 20 without data augmentation
- lr = 3e-2
- conv 1x1 = 50 filers
- dense = 100
- Dropout = 0.5

Epoch 20/20
713/713 [==============================] - 385s - loss: 1.9519 - acc: 0.0358 - mean_absolute_error: 9.6799 - val_loss: 4.6270 - val_acc: 0.0444 - **val_mean_absolute_error: 9.0526**

